In [1]:
import src.load_data as load_data
from src.feature_matrix import create_feature_matrix
import seaborn as sns
import pandas as pd

# load in the data
joined = load_data.data_pipeline()
feature_matrix = create_feature_matrix(joined, melody_note_lags=range(1,5), chord_root_lags=range(-1,-2,-1), chord_type_lags=range(-1,-2,-1))

In [2]:
y = feature_matrix['chord_root_melody_note_interval']
X = feature_matrix.drop(['chord_root_melody_note_interval'], axis=1)